In [183]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt

pd.options.plotting.backend = "matplotlib"
pd.options.plotting.backend = "plotly"
variant: int = 9

In [184]:
df: pd.DataFrame = pd.read_excel(r"raw_data.xlsx",
                                 dtype={"Tel_fil": str,
                                        "user_code": str})

df["start_time"] = pd.to_timedelta(df["start_time"].astype(str))
df["start_time"] = pd.to_timedelta(df["start_time"])
df["start_time"] = df["date"] + df["start_time"]

df["duration"] = df["duration"] + variant * 0.1
df["cost"] = df["duration"] * 5

df["end_time"] = df["start_time"] + pd.to_timedelta(df["duration"], unit="m")

df.drop(columns=["date"], inplace=True)
df

,branch_code,user_code,user_region,start_time,duration,cost,end_time
0,236880,962580,САХАЛИН МОБИЛЬНЫЙ,2025-02-02 08:37:00,1.9,9.5,2025-02-02 08:38:54
1,236880,91475,ЮЖНО-САХАЛИНСК,2025-02-02 12:53:00,1.9,9.5,2025-02-02 12:54:54
2,236880,91464,САХАЛИНСКАЯ ОБЛ,2025-02-02 13:42:00,1.9,9.5,2025-02-02 13:43:54
3,236880,962580,САХАЛИН МОБИЛЬНЫЙ,2025-02-02 14:30:00,1.9,9.5,2025-02-02 14:31:54
4,236880,962580,САХАЛИН МОБИЛЬНЫЙ,2025-02-03 08:18:00,1.9,9.5,2025-02-03 08:19:54
...,...,...,...,...,...,...,...
153,236880,4212,ХАБАРОВСК,2025-02-26 15:23:00,6.9,34.5,2025-02-26 15:29:54
154,236880,91415,ХАБАРОВСК МОБИЛЬНЫЙ,2025-02-26 15:45:00,3.9,19.5,2025-02-26 15:48:54
155,236880,4212,ХАБАРОВСК,2025-02-27 10:53:00,1.9,9.5,2025-02-27 10:54:54
156,236880,4212,ХАБАРОВСК,2025-02-27 11:05:00,2.9,14.5,2025-02-27 11:07:54


In [185]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158 entries, 0 to 157
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   branch_code  158 non-null    int64         
 1   user_code    158 non-null    object        
 2   user_region  158 non-null    object        
 3   start_time   158 non-null    datetime64[ns]
 4   duration     158 non-null    float64       
 5   cost         158 non-null    float64       
 6   end_time     158 non-null    datetime64[ns]
dtypes: datetime64[ns](2), float64(2), int64(1), object(2)
memory usage: 8.8+ KB


In [186]:
df.describe()

,branch_code,start_time,duration,cost,end_time
count,158.0,158,158.000000,158.000000,158
mean,236880.0,2025-02-15 13:44:53.544303872,3.216456,16.082278,2025-02-15 13:48:06.531645952
min,236880.0,2025-02-02 08:37:00,1.900000,9.500000,2025-02-02 08:38:54
25%,236880.0,2025-02-05 19:26:00,1.900000,9.500000,2025-02-05 19:27:54
50%,236880.0,2025-02-17 09:20:00,1.900000,9.500000,2025-02-17 09:21:54
75%,236880.0,2025-02-24 12:43:30,3.900000,19.500000,2025-02-24 12:45:24
max,236880.0,2025-02-27 15:14:00,13.900000,69.500000,2025-02-27 15:15:54
std,0.0,NaN,2.147695,10.738475,NaN


In [187]:
df[["user_region", "user_code"]]

,user_region,user_code
0,САХАЛИН МОБИЛЬНЫЙ,962580
1,ЮЖНО-САХАЛИНСК,91475
2,САХАЛИНСКАЯ ОБЛ,91464
3,САХАЛИН МОБИЛЬНЫЙ,962580
4,САХАЛИН МОБИЛЬНЫЙ,962580
...,...,...
153,ХАБАРОВСК,4212
154,ХАБАРОВСК МОБИЛЬНЫЙ,91415
155,ХАБАРОВСК,4212
156,ХАБАРОВСК,4212


In [188]:
df["weekday_name"] = df["start_time"].dt.day_name(locale="Russian")
df.groupby("weekday_name").size().sort_values(ascending=False).plot(kind="bar")

In [189]:
df["hour"] = df["start_time"].dt.hour
heatmap_data = df.groupby(["weekday_name", "hour"]).size().reset_index(name="calls")
fig = px.imshow(
    heatmap_data.pivot(index="weekday_name", columns="hour", values="calls"),
     labels=dict(x="Час суток", y="День недели", color="Количество звонков"),
)

fig.show()


In [190]:
heatmap_data = (
    df.groupby(["weekday_name", "hour"])["duration"]
      .mean()
      .reset_index()
)
pivot = heatmap_data.pivot(index="weekday_name", columns="hour", values="duration")
fig = px.imshow(pivot, aspect="auto",
    color_continuous_scale="YlGnBu")
fig.show()

In [191]:
df.groupby("user_code").size().sort_values(ascending=False).plot(kind="bar")


In [192]:
df.groupby("user_region").size().sort_values(ascending=False).plot(kind="bar")

In [ ]:
df[["user_region", "duration", "cost"]].groupby("user_region").aggregate("mean").sort_values("cost", ascending=False).plot(kind="bar")

,duration,cost
user_region,,
САХАЛИН МОБИЛЬН,4.900000,24.500000
САХАЛИН МОБИЛЬНЫЙ,2.742105,13.710526
САХАЛИНСКАЯ ОБЛ,2.163158,10.815789
САХАЛИНСКАЯ ОБЛ МОБИЛЬНЫЙ,1.900000,9.500000
ТЫМОВСК,3.445455,17.227273
ХАБАРОВСК,3.633333,18.166667
ХАБАРОВСК МОБИЛ,1.900000,9.500000
ХАБАРОВСК МОБИЛЬНЫЙ,3.900000,19.500000
ХАБАРОВСКИЙ КР,3.900000,19.500000


In [194]:
df.groupby("user_region")[["duration", "cost"]].sum().sort_values("cost", ascending=False).plot(kind="bar")


In [195]:
df[["user_code", "duration", "cost"]].groupby("user_code").aggregate("mean").sort_values("cost", ascending=False).plot(kind="bar")

In [199]:
df[["user_code", "duration", "cost"]].groupby("user_code").aggregate(func="sum").sort_values("cost", ascending=False).plot(kind="bar")

In [196]:
px.scatter(df, "cost", "duration")

In [202]:
agg_stats = (
    df.groupby("user_region")[["duration", "cost"]]
      .agg(["count", "mean", "sum"])
      .sort_values(("cost", "sum"), ascending=False)
)

agg_stats


duration                   cost                   
                             count      mean    sum count       mean     sum
user_region                                                                 
ХАБАРОВСК                       75  3.633333  272.5    75  18.166667  1362.5
САХАЛИН МОБИЛЬНЫЙ               38  2.742105  104.2    38  13.710526   521.0
САХАЛИНСКАЯ ОБЛ                 19  2.163158   41.1    19  10.815789   205.5
ТЫМОВСК                         11  3.445455   37.9    11  17.227273   189.5
ЮЖНО-САХАЛИНСК МОБИЛЬНЫЙ         5  4.900000   24.5     5  24.500000   122.5
ЮЖНО-САХАЛИНСК                   5  2.300000   11.5     5  11.500000    57.5
САХАЛИН МОБИЛЬН                  1  4.900000    4.9     1  24.500000    24.5
ХАБАРОВСК МОБИЛЬНЫЙ              1  3.900000    3.9     1  19.500000    19.5
ХАБАРОВСКИЙ КР                   1  3.900000    3.9     1  19.500000    19.5
САХАЛИНСКАЯ ОБЛ МОБИЛЬНЫЙ        1  1.900000    1.9     1   9.500000     9.5
ХАБАРОВСК МОБИЛ                  1  1.900000    1.9     1   9.500000     9.5

In [ ]:
df[["cost", "duration"]].corr()

,cost,duration
cost,1.0,1.0
duration,1.0,1.0
